In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import json
import requests

In [2]:
import tweepy
from tweepy import Client

key = 'B81Sj4ifYRstLZGMyMDM0roG3'
secret = 'c1Rg4vdyl2mjMEC7WKDsVLTz2IlqTHy1yezn71Y9cw3LgZaxAW'
token = 'AAAAAAAAAAAAAAAAAAAAAD6eWQEAAAAABtT80i7lkSemhYsuOIjy4O4xLQs%3DkfsByhRoXQ9odyT51sECR1vXxOLKxdtOON2Ij4OODdAt8jTcGY'

api = tweepy.Client(bearer_token=token, consumer_key=key, consumer_secret=secret, 
              return_type=requests.Response,
              wait_on_rate_limit=True)

In [3]:
combined = pd.read_pickle('node_and_degree_list.pkl')
combined.head()

,twitter_id,followers_count,following_count,label
0,461277906,380850,2913,bot
1,808280162,23363,340,bot
2,1575510320,120435,108954,bot
3,2449235599,25581,16,bot
4,1113769100,73720,77585,bot


In [4]:
# Set follower/following threshold
thresh = 1000

print('BEFORE FILTERING')
print(f'Total accounts: {combined.shape[0]}')
nb = combined.loc[combined.label == 'bot'].shape[0]
nh = combined.loc[combined.label == 'human'].shape[0]
print(f'Bot accounts: {nb}')
print(f'Human accounts: {nh}')
print('____________________')


# Drop accounts that exceed threshold
small = combined.loc[combined.followers_count < thresh]
small = small.loc[small.followers_count < thresh]

print('AFTER FILTERING')
print(f'Total accounts: {small.shape[0]}')
nb = small.loc[small.label == 'bot'].shape[0]
nh = small.loc[small.label == 'human'].shape[0]
print(f'Bot accounts: {nb}')
print(f'Human accounts: {nh}')

BEFORE FILTERING
Total accounts: 2263
Bot accounts: 959
Human accounts: 1304
____________________
AFTER FILTERING
Total accounts: 458
Bot accounts: 193
Human accounts: 265


In [5]:
# Small df contains accounts meeting threshold criteria
small.head()

,twitter_id,followers_count,following_count,label
24,4307623643,59,4,bot
36,844779426260791296,768,1,bot
83,837316817324556289,182,6,bot
137,913028718377005056,87,147,bot
175,790017240733278208,19,47,bot


In [6]:
# Create curr_edgelist for a network including nodes up to 3 degrees of separation (3 links away)
def add_this_nodes_edgelist(this_acc, curr_edgelist):
    next_page_token = None
    visited.add(this_acc)

    action = 'followers'
    follow_list = []
    this_acc_num_following = []    
    this_acc_num_followers = []
    while True:
        # Get immediate followers/following of node and add these to curr_edgelist
        if action == 'followers':
            request = api.get_users_followers(id=this_acc, user_auth=False, 
                max_results=1000, pagination_token=next_page_token, user_fields=['public_metrics'])
        elif action == 'following':
            request = api.get_users_following(id=this_acc, user_auth=False, 
                max_results=1000, pagination_token=next_page_token, user_fields=['public_metrics'])

        data = request.json()

        # for each account following/followed by the originating account, record account id, 
        # num following, and num followed by
        try:
            for acc in data['data']:
                follow_list.append(acc['id'])
                num_following = acc['public_metrics']['following_count']
                num_followed_by = acc['public_metrics']['followers_count']
                this_acc_num_following.append(num_following)
                this_acc_num_followers.append(num_followed_by)

            this_acc_list = [this_acc for i in range(len(follow_list))]

        # Catch errors if not authorized to view user
        except Exception as e:
            print('Exception: issue retrieving follow lists for this account -- Skipping.')
            return curr_edgelist
            
            
        if 'next_token' in data['meta'].keys():    # get page token for next request
                next_page_token = data['meta']['next_token']
        
        # when all data recorded transition to next action
        else:
            next_page_token = None
            if action == 'followers':
                d = {'originating_id': follow_list,
                    'receiving_id': this_acc_list,
                    'originating_following_count': this_acc_num_following, 
                    'originating_follower_count': this_acc_num_followers}
                edges_to_add = pd.DataFrame(d)
                curr_edgelist = pd.concat([curr_edgelist, edges_to_add])

                action = 'following'

                # reset lists
                follow_list = []
                this_acc_num_following = []    
                this_acc_num_followers = []

            elif action == 'following':
                d = {'originating_id': this_acc_list,
                    'receiving_id': follow_list,
                    'receiving_following_count': this_acc_num_following, 
                    'receiving_follower_count': this_acc_num_followers}
                edges_to_add = pd.DataFrame(d)
                curr_edgelist = pd.concat([curr_edgelist, edges_to_add])

                next_page_token = None

                return curr_edgelist

In [7]:
def transition_to_next_separation_degree(curr_edgelist, degrees_of_separation):
    try:
        # Drop all nodes with followers or following more than threshold
        to_drop = (curr_edgelist.originating_following_count > thresh) | \
            (curr_edgelist.originating_follower_count > thresh) | \
            (curr_edgelist.receiving_following_count > thresh) | \
            (curr_edgelist.receiving_follower_count > thresh)
        curr_edgelist.drop(index = curr_edgelist[to_drop].index, inplace=True)

        # create list of all neighbors in curr_edgelist df
        unvisited_neighbors = list(curr_edgelist.originating_id.append(curr_edgelist.receiving_id).unique())

        # if any nodes are in visited set, remove
        [unvisited_neighbors.remove(i) for i in unvisited_neighbors if i in visited]

        print()
        print(f'Current Number of edges from starting node {degrees_of_separation}')
        print(f'Num unvisited neighbors: {len(unvisited_neighbors)}')
        print('--------------------')
        degrees_of_separation +=1 
    
        return curr_edgelist, degrees_of_separation, unvisited_neighbors
    
    
    except Exception as e:
        print(e)
        print('--------------------EXCEPTION--------------------')
        return curr_edgelist, degrees_of_separation, []
        
    

In [8]:
import os

# List of account ids that I already downloaded edgelist for
already_processed = []
 
for name in os.listdir('human'):
    if os.path.isdir(name):
        continue
    list_name = name.replace('.pkl','').replace('edgelist_2_deg_','')
    already_processed.append(int(list_name))

In [10]:
cols=['originating_id', 'receiving_id', 
          'originating_following_count', 'originating_follower_count',
         'receiving_following_count', 'receiving_follower_count']

# Process human accounts 
small_human_accs = small.loc[small.label == 'human']
small_human_accs['sum_follows'] = small_human_accs['followers_count'] + small_human_accs['following_count']
small_human_accs.sort_values('sum_follows',inplace=True)
small_human_accs.head()

# Create Edgelists for each account in the filtered botometer dataset.
for starting_acc, label in zip(small_human_accs.twitter_id, small_human_accs.label):
    if starting_acc in already_processed:
        continue
    print(f'Starting node == {starting_acc}')
    # Initialization code for populating edgelist starting from node specified by "starting_acc"
    edgelist = pd.DataFrame(columns=cols)

    degrees_of_separation = 0

    visited = set()
    visited.add(starting_acc)

    edgelist = add_this_nodes_edgelist(starting_acc, edgelist) 
    edgelist, degrees_of_separation, unvisited_neighbors = transition_to_next_separation_degree(edgelist, degrees_of_separation)
    
    # iterate thru all unvisited neighbors with 2 degree of separation from starting (labeled) node
    # process df in chunks before concatenating
    while degrees_of_separation <= 1:
        curr_edgelist = pd.DataFrame(columns=cols)
        for i in unvisited_neighbors:
            print(i)
            curr_edgelist = add_this_nodes_edgelist(i, curr_edgelist)
            visited.add(i)
        
        curr_edgelist, degrees_of_separation, unvisited_neighbors = transition_to_next_separation_degree(curr_edgelist, degrees_of_separation)
        edgelist = pd.concat([edgelist, curr_edgelist])

    # Save in directory
    fn = 'edgelist_2_deg_' + str(starting_acc) + '.pkl'
    path = str(label) + '/' + fn
    edgelist.to_pickle(path)
    
    # Add twitter id to already processed list
    already_processed.append(starting_acc)

<ipython-input-10-504fdcfe9d1b>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small_human_accs['sum_follows'] = small_human_accs['followers_count'] + small_human_accs['following_count']
<ipython-input-10-504fdcfe9d1b>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small_human_accs.sort_values('sum_follows',inplace=True)


Starting node == 938469204021469184

Current Number of edges from starting node 0
Num unvisited neighbors: 111
--------------------
709901980685221888
1409221457893330949
1274319430504284161
1328836555792904195
1319010483094102017
1729500662
Exception: issue retrieving follow lists for this account -- Skipping.
1206730070443921410
1282302030770581504
259801552
869321628995518465
Exception: issue retrieving follow lists for this account -- Skipping.
30955603
1445122124
793004240822161408
245760942
Exception: issue retrieving follow lists for this account -- Skipping.
4670837555


Rate limit exceeded. Sleeping for 788 seconds.


Exception: issue retrieving follow lists for this account -- Skipping.
1003788354532577282
Exception: issue retrieving follow lists for this account -- Skipping.
129197865
1084475466
237043305
Exception: issue retrieving follow lists for this account -- Skipping.
885309568506580993
1044046729815875584


Rate limit exceeded. Sleeping for 5 seconds.


...

Rate limit exceeded. Sleeping for 892 seconds.


999332693656600578
999325926377828358
953274077334122496
989265286401490944
932916400171167744
3041610976
996410397266767872
990374329232908288
Exception: issue retrieving follow lists for this account -- Skipping.
171183395
Exception: issue retrieving follow lists for this account -- Skipping.
2564359489
951805632105123840
988827522229067777
Exception: issue retrieving follow lists for this account -- Skipping.
43627796

Current Number of edges from starting node 1
Num unvisited neighbors: 0
--------------------
Starting node == 953231816810315777


Rate limit exceeded. Sleeping for 885 seconds.



Current Number of edges from starting node 0
Num unvisited neighbors: 64
--------------------
1072495536194945028


KeyboardInterrupt: 